In [1]:
# Import libraries used in this notebook
import os
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

In [3]:
# Write function to load PRTSZ transmission percent from file name
def load_prtsz_transmission(prtsz_file):
    """ Import PRTSZ data from CSV, use date and time
    columns to create Pandas datetime Index, and define
    Xarray Dataset object containing total volume
    concentration time series.
    Author: Kylene Cooley, 19 Apr 2024
    Inputs:
    -------
    prtsz_file, str of file name in local PRTSZ along shelf CTD 
        survey data directory
    Returns:
    --------
    vol_con, Data Array of total volume concentration from PRTSZ
        dataset with 1 dim = "time"
    """
    # Define path to locally saved PRTSZ data from CN site
    prtsz_path = os.path.join(os.path.expanduser(
    "~/Desktop/pioneer-20_data/PRTSZ_on_rosette/AR82b_CAST003/"),
    prtsz_file)
    # Import csv file using pandas and define variables
    prtsz_data = pd.read_csv(prtsz_path, header=None)
    transmission = prtsz_data[59]
    depth = prtsz_data[40]
    year = prtsz_data[42]
    month = prtsz_data[43]
    day = prtsz_data[44]
    hour = prtsz_data[45]
    minute = prtsz_data[46]
    second = prtsz_data[47]
    # Convert time parameter series to datetime vector
    dt_df = pd.DataFrame({"year": year, "month": month,
                        "day": day, "hour": hour,
                        "minute": minute, "second": second})
    prtsz_dt = pd.to_datetime(dt_df,
                            yearfirst=True, utc=True)
    # Create data array of total volume concentration with series
    prtsz_transmission = xr.Dataset(data_vars=dict(transmission=(["time"], transmission),
                                    depth=(["time"], depth)),
                            coords={"time": prtsz_dt}
                            )
    return prtsz_transmission

In [4]:
# Load CTD cast #003 from AR82-B
prtsz_file = "ar82b_cast003_1071418_rs.csv"
prtsz_transmission = load_prtsz_transmission(prtsz_file)
prtsz_transmission

In [6]:
# Filter the full data set for depths where the PRTSZ
# was submerged below the sea surface according to depth
prtsz_water = prtsz_transmission.where(
    prtsz_transmission.depth>0, drop=True)
prtsz_water

<xarray.Dataset>
Dimensions:       (time: 616)
Coordinates:
  * time          (time) datetime64[ns] 2024-04-16T14:22:10 ... 2024-04-16T14...
Data variables:
    transmission  (time) float64 0.942 0.954 0.951 0.924 ... 0.951 0.956 0.954
    depth         (time) float64 0.36 1.63 1.73 1.86 ... 1.26 1.67 1.55 0.39

In [7]:
# Evaluate mean percent transmission for data recorded below
# the sea surface as recorded by the PRTSZ 
prtsz_water.transmission.mean()

<xarray.DataArray 'transmission' ()>
array(0.96008766)

In [8]:
# Evaluate mean percent transmission for all data recorded
prtsz_transmission.transmission.mean()

<xarray.DataArray 'transmission' ()>
array(0.93800496)

In [9]:
# Check average percent transmission when PRTSZ is above the surface.
# The times with percent transmission less than 30% were recorded 
# after the CTD rosette came out of the water.
prtsz_transmission.transmission.where(prtsz_transmission.depth<=0, drop=True).mean()

<xarray.DataArray 'transmission' ()>
array(0.86678534)